In [19]:
import numpy as np 
from random import shuffle
import pandas as pd
from glob import glob

In [20]:
spot_images = glob("./data/spot_top/*.csv")
spot_images = [s.split('_')[-1] for s in spot_images]
spot_images = np.unique(spot_images).tolist()
aspots = {'top':{},
          'mid':{},
          'bot':{}}
for s in spot_images:
    aspots['top'][s.split('.')[0]] = glob("./data/spot_top/*"+s)
    aspots['mid'][s.split('.')[0]] = glob("./data/spot_mid/*"+s)
    aspots['bot'][s.split('.')[0]] = glob("./data/spot_bot/*"+s)

In [21]:
print(aspots)

{'top': {'beach': ['./data/spot_top/U1C_U6C_beach.csv', './data/spot_top/U3A_U4C_beach.csv', './data/spot_top/U3C_U4B_beach.csv', './data/spot_top/U2B_U5C_beach.csv', './data/spot_top/U7B_U12B_beach.csv', './data/spot_top/U7C_U12C_beach.csv', './data/spot_top/U2C_U5A_beach.csv'], 'church': ['./data/spot_top/U7C_U12C_church.csv', './data/spot_top/U7B_U12B_church.csv', './data/spot_top/U3C_U4A_church.csv', './data/spot_top/U1A_U6C_church.csv', './data/spot_top/U3B_U4C_church.csv'], 'city': ['./data/spot_top/U3C_U4A_city.csv', './data/spot_top/U7C_U12C_city.csv', './data/spot_top/U3B_U4C_city.csv', './data/spot_top/U7B_U12B_city.csv', './data/spot_top/U3A_U4B_city.csv'], 'farm': ['./data/spot_top/U7C_U12C_farm.csv', './data/spot_top/U1C_U6C_farm.csv', './data/spot_top/U3C_U4B_farm.csv', './data/spot_top/U1A_U6A_farm.csv'], 'house': ['./data/spot_top/U2C_U5B_house.csv', './data/spot_top/U3B_U4B_house.csv', './data/spot_top/U7C_U12C_house.csv', './data/spot_top/U7B_U12B_house.csv', './data/

In [22]:
print(spot_images)
spot_images = set(spot_images)
print(spot_images)

['beach.csv', 'church.csv', 'city.csv', 'farm.csv', 'house.csv', 'jungle.csv', 'sea.csv', 'sheep.csv', 'winter.csv']
{'beach.csv', 'church.csv', 'sea.csv', 'winter.csv', 'jungle.csv', 'house.csv', 'farm.csv', 'city.csv', 'sheep.csv'}


In [23]:
def generate_successives(content, aspots, spot_images, generated, n):
    choice = np.random.choice(list(spot_images - set(content)), n, replace=False) 
    choice = [np.random.choice(aspots["mid"][c.split('.')[0]]) for c in choice[:-1]] + [np.random.choice(aspots["bot"][choice[0].split('.')[0]])]
    temp_content = [content] + choice
    str_temp_content = ""
    for c in temp_content:
        str_temp_content += c

    cpt = 0
    while str_temp_content in generated:
        choice = np.random.choice(list(spot_images - set(content)), n, replace=False) 
        choice = [np.random.choice(aspots["mid"][c.split('.')[0]]) for c in choice[:-1]] + [np.random.choice(aspots["bot"][choice[0].split('.')[0]])]
        temp_content = [content] + choice
        str_temp_content = ""
        for c in temp_content:
            str_temp_content += c
        if cpt > 100:
            return -1 
        else :
            cpt += 1
    generated += [str_temp_content]
    return list(choice)


In [24]:
from ast import literal_eval
def add_offset(serie, offset):
    #print("OFFSET :", offset)
    #print(serie)
    res = []
    for i,row in enumerate(serie):
        if type(row) == str:
            row = literal_eval(row)
            if row != []:
                res += [[v+(100*offset) for v in row]]
            else :
                res+=[]
        else: 
            res += [np.nan]
    #print(pd.Series(res))
    return pd.Series(res)

In [25]:
transitions = [
    "Moving on to the next picture.",
    "Proceeding to the next photo.",
    "Moving ahead to the next image.",
    "Let's jump to the next image.",
    "Let's see the next image now.",
    "Let's look at the next image.",
    "Let's go to the next image now.",
    "Let's go next.",
    "Let's check out the next image.",
    "Time to move to the next picture.",
    "Let's switch to the next image now.",
    "Let's proceed to the next image."
]
    
def add_transition():
    #if np.random.random() > 0.5:
        #return np.random.choice(transitions_directed)
    #else :
        return np.random.choice(transitions)

In [26]:
def combine_spot(n, t):
    spot = glob("./data/spot_top/*.csv")
    spot = [s.split("/")[-1] for s in spot]
    
    generated = []
    dataset = []
    for i in range(t):
        dfs = []
        
        content = set()
        top = i % len(spot)
        top_df = pd.read_csv("./data/spot_top/"+spot[top])
        names = spot[top].split("/")[-1].split(".")[0] + '-'
        dfs += [top_df]
        dfs[-1] = pd.concat([dfs[-1], dfs[-1].iloc[-1:]], ignore_index=True)
        dfs[-1].loc[dfs[-1].index[-1], "utterance"] = add_transition()
        content = content.union(set([spot[top].split('_')[-1]]))
        next_dialogues = generate_successives(spot[top], aspots, spot_images, generated, n-1)
        #print(next_dialogues)
        for j in range(n-1):
            mid_df = pd.read_csv(next_dialogues[j])
            mid_df.loc[:, "DU ID"]  = add_offset(mid_df["DU ID"], j+1)
            mid_df.loc[:, "Grounded CGUs"]  = add_offset(mid_df["Grounded CGUs"], j+1)
            mid_df.loc[:, "Opened CGUs"]  = add_offset(mid_df["Opened CGUs"], j+1)
            if j != n-2 : 
                names += next_dialogues[j].split("/")[-1].split(".")[0] + '-'
            else : 
                names += next_dialogues[j].split("/")[-1].split(".")[0]
            dfs += [mid_df]
            dfs[-1] = pd.concat([dfs[-1], dfs[-1].iloc[-1:]], ignore_index=True)
            dfs[-1].loc[dfs[-1].index[-1], "utterance"] = add_transition()

        #print(names)
        dfs = pd.concat(dfs)
        path_save = "./data/merged_spot/merge_" + names + ".csv"
        dfs.to_csv(path_save, index=False)
        dataset += [dfs]
    return dataset
datatest = combine_spot(4,150)